In [69]:
import pandas as pd
import numpy as np
import cv2
import os
import re

In [70]:
bodiesDF = pd.read_pickle('synchronization/synchronize')
facesDF = pd.read_pickle('faces')
uniqueNames = np.load('uniqueName.npy')

In [71]:
scores = [[] for i in range(len(uniqueNames))]

In [72]:
facesDF

,Laurene,Anthony,Wissem,Cynthia,Sabrine,faceX,faceY,rgbSeq,blur
0,1.341175,1.302884,1.315100,1.046635,1.339248,456.0,405.0,0.0,663.778234
1,1.297213,1.228618,1.401256,0.710368,1.398127,423.0,409.0,3.0,395.108355
2,1.257040,1.269675,1.339069,0.772881,1.326457,417.0,415.0,5.0,275.320893
3,1.270489,1.265388,1.339574,0.756341,1.361371,422.0,416.0,6.0,291.272770
4,1.159458,1.226889,1.274342,0.774507,1.388032,426.0,425.0,7.0,398.074038
5,1.230790,1.286956,1.278533,0.781219,1.344135,443.0,437.0,8.0,555.155143
6,1.206173,1.248794,1.242732,0.760098,1.361885,465.0,438.0,9.0,424.413457
7,1.207176,1.250816,1.254134,0.807621,1.354820,500.0,428.0,11.0,512.477635
8,1.238172,1.225516,1.232129,0.800250,1.347640,517.0,428.0,12.0,542.142234
9,1.267545,1.220619,1.211027,0.813766,1.321908,551.0,428.0,13.0,557.726855


In [73]:
for filename in os.listdir('/home/sabrine/notebook/modified2'):
    img = cv2.imread("/home/sabrine/notebook/modified2/" + filename)
    
    rgbSeq = re.search('\d+',filename).group(0)

    for index, row in facesDF[facesDF.rgbSeq == int(rgbSeq)].iterrows():
        center = (int(row.faceX), int(row.faceY))
        radius = 10
        color = (64, 255, 64);
        thickness = 2
        lineType = 8 # will discorver what it means
        img = cv2.circle(img, center, radius, color, thickness, lineType)
        
    cv2.imwrite("/home/sabrine/notebook/modified/" + filename, img)    

In [104]:
for seq in bodiesDF.rgbSeq.unique():
    #seq = 14
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgbSeq == seq]
    
    img = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + ".jpg")
    
    for index, face in faces.iterrows():
        faceCentroid = (int(face.faceX), int(face.faceY))
        
        for i, b in bodies.iterrows():
            upLeft = tuple((int(b.trackX), int(b.trackY)))
            downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
            bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
            
            inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2
            up = bodyCentroid[1] > faceCentroid[1]
            if inside and up:
                color = (255, 255, 64)
            else:
                color = (64, 255, 255)
            thickness = 1
            lineType = 8 # will discorver what it means
            radius = b.trackH.astype(int)/2
            img = cv2.circle(img, faceCentroid, int(radius), color, thickness, lineType)
    cv2.imwrite("/home/sabrine/notebook/modified/" + str(seq) + ".jpg", img)


In [55]:
bodiesDF.head(50)

traSeq  trackId  trackX  trackY  trackW  trackH    height  \
seq   id                                                                
784.0 12.0   784.0     12.0   238.0     278     396     792  1.597804   
      14.0   784.0     14.0    81.6     400     108     216  0.803921   
785.0 12.0   785.0     12.0   217.6     276     404     812  1.598425   
      14.0   785.0     14.0    81.6     400     108     216  0.803921   
786.0 12.0   786.0     12.0   193.8     276     420     840  1.599651   
      14.0   786.0     14.0    81.6     400     108     216  0.803921   
787.0 12.0   787.0     12.0   170.0     276     432     864  1.598773   
      14.0   787.0     14.0    81.6     400     108     216  0.803921   
788.0 12.0   788.0     12.0   163.2     278     436     872  1.598104   
      14.0   788.0     14.0    81.6     400     108     216  0.803921   
789.0 12.0   789.0     12.0   156.4     280     440     880  1.597755   
      14.0   789.0     14.0    81.6     400     108     216  0.803921   
790.0 12.0   790.0     12.0   146.2     282     444     892  1.597284   
      14.0   790.0     14.0    81.6     400     108     216  0.803921   
791.0 12.0   791.0     12.0   142.8     284     448     900  1.597968   
      14.0   791.0     14.0    81.6     400     108     216  0.803921   
792.0 12.0   792.0     12.0   122.4     284     464     928  1.596457   
      14.0   792.0     14.0    81.6     400     108     216  0.803921   
793.0 12.0   793.0     12.0   112.2     284     476     956  1.594237   
      14.0   793.0     14.0    81.6     400     108     216  0.803921   
794.0 12.0   794.0     12.0   115.6     286     488     976  1.592291   
      14.0   794.0     14.0    81.6     400     108     216  0.803921   
795.0 12.0   795.0     12.0   136.0     288     492     988  1.590862   
      14.0   795.0     14.0    81.6     400     108     216  0.803921   
796.0 12.0   796.0     12.0   146.2     288     496     996  1.592582   
      14.0   796.0     14.0    81.6     400     108     216  0.803921   
797.0 12.0   797.0     12.0   159.8     290     504    1012  1.594597   
      14.0   797.0     14.0    81.6     400     108     216  0.803921   
798.0 12.0   798.0     12.0   180.2     290     520    1040  1.595444   
      14.0   798.0     14.0    81.6     400     108     216  0.803921   
799.0 12.0   799.0     12.0   210.8     290     528    1056  1.597045   
      14.0   799.0     14.0    81.6     400     108     216  0.803921   
800.0 12.0   800.0     12.0   238.0     292     532    1064  1.598122   
      14.0   800.0     14.0    81.6     400     108     216  0.803921   
801.0 12.0   801.0     12.0   272.0     292     536    1076  1.598573   
      14.0   801.0     14.0    81.6     400     108     216  0.803921   
802.0 12.0   802.0     12.0   312.8     294     540    1080  1.599119   
      14.0   802.0     14.0    81.6     400     108     216  0.803921   
806.0 12.0   806.0     12.0   442.0     294     540    1084  1.601227   
      14.0   806.0     14.0    81.6     400     108     216  0.803921   
807.0 12.0   807.0     12.0   489.6     296     536    1076  1.603713   
      14.0   807.0     14.0    81.6     400     108     216  0.803921   
808.0 12.0   808.0     12.0   510.0     298     532    1068  1.603847   
      14.0   808.0     14.0    81.6     400     108     216  0.803921   
809.0 12.0   809.0     12.0   547.4     300     528    1060  1.603456   
      14.0   809.0     14.0    81.6     400     108     216  0.803921   
810.0 12.0   810.0     12.0   622.2     302     520    1044  1.603647   
811.0 12.0   811.0     12.0   642.6     304     516    1036  1.602410   
812.0 12.0   812.0     12.0   659.6     306     512    1024  1.599830   
813.0 12.0   813.0     12.0   717.4     308     500    1004  1.597472   

            distance  rgbSeq  
seq   id                      
784.0 12.0  2.993590       0  
      14.0  5.991756       0  
785.0 12.0  2.921760       0  
      14.0  5.991756       0  
786.0 12.0  2.825866       3 

In [30]:
for seq in bodiesDF.rgbSeq.unique():
    bodies = bodiesDF[bodiesDF.rgbSeq == seq]
    faces = facesDF[facesDF.rgbSeq == seq]
    
    #print 'bodies'
    #print bodies
    #print '####'
    #print 'faces'
    #print faces
    for i, f in faces.iterrows():
        if len(bodies) > 0 :        
            
            candidates = bodies[
                [((((bodies.trackX.astype(int) + bodies.trackW.astype(int)) / 2 ) - f.faceX)**2 + 
                (((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) - f.faceY)**2) < 
                 (bodies.trackH.astype(int)/2)**2] # 250 arbitrary picked 
                and 
                ((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) < f.faceY
            ]
            if not candidates.empty:
                print '******'
                print f
                print candidates
                print '******'

******
Laurene      1.341175
Anthony      1.302884
Wissem       1.315100
Cynthia      1.046635
Sabrine      1.339248
faceX      456.000000
faceY      405.000000
rgbSeq       0.000000
blur       663.778234
Name: 0, dtype: float64
            traSeq  trackId  trackX  trackY  trackW  trackH    height  \
seq   id                                                                
784.0 14.0   784.0     14.0    81.6     400     108     216  0.803921   
785.0 14.0   785.0     14.0    81.6     400     108     216  0.803921   

            distance  rgbSeq  
seq   id                      
784.0 14.0  5.991756       0  
785.0 14.0  5.991756       0  
******
******
Laurene      1.297213
Anthony      1.228618
Wissem       1.401256
Cynthia      0.710368
Sabrine      1.398127
faceX      423.000000
faceY      409.000000
rgbSeq       3.000000
blur       395.108355
Name: 1, dtype: float64
            traSeq  trackId  trackX  trackY  trackW  trackH    height  \
seq   id                                     

In [6]:
bodiesDF

,,traSeq,trackId,trackX,trackY,trackW,trackH,height,distance,rgbSeq
seq,is,,,,,,,,,
1155,4,1155,4,233,0,288,576,1.5005300045,1.42739236355,42
1156,4,1156,4,272,13,270,541,1.50847764015,1.56097233295,45
1157,4,1157,4,292,19,262,524,1.5179650116,1.58523178101,47
1158,4,1158,4,334,33,243,487,1.53235393715,1.71000134945,48
1159,4,1159,4,368,44,228,457,1.55183779554,1.81243562698,51
1160,4,1160,4,415,62,206,413,1.57554826161,1.9273519516,52
1161,4,1161,4,423,62,203,406,1.59748577792,2.0393819809,52
1162,4,1162,4,434,67,196,392,1.61563918339,2.07068181038,53
1163,4,1163,4,442,68,192,384,1.63903561843,2.18748092651,54


In [7]:
bodies

,,traSeq,trackId,trackX,trackY,trackW,trackH,height,distance,rgbSeq
seq,is,,,,,,,,,
1157,4,1157,4,292,19,262,524,1.5179650116,1.58523178101,47
